In [1]:
import nltk
nltk.download('all',halt_on_error=False)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [0]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
% matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#nltk
import nltk

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for part-of-speech tagging
from nltk import pos_tag

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#model_selection
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#evaluation
from sklearn.metrics import accuracy_score,roc_auc_score 
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_confusion_matrix

#preprocessing scikit
from sklearn.preprocessing import MinMaxScaler,StandardScaler,Imputer,LabelEncoder

#modelling algos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier

#gensim w2v
#word2vec
from gensim.models import Word2Vec

#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
train=pd.read_csv(r'drive/Colab Notebooks/spooky author/train.csv')
test=pd.read_csv(r'drive/Colab Notebooks/spooky author/test.csv')

In [0]:
df=train.copy()

In [7]:
print(df.shape)
df.head()

(19579, 3)


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [8]:
print(df.author.isnull().sum())  # no null values
df.text.isnull().sum()

0


0

In [9]:
print(test.shape)
test.head()

(8392, 2)


,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [10]:
# checking null values in test set.
print(test.text.isnull().sum())  # no null values
 

0


In [11]:
# back to training set.
df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [0]:
# drop the id column.
df.drop(['id'],axis=1,inplace=True)

In [13]:
df.head(10)

,text,author
0,"This process, however, afforded me no means of...",EAP
1,It never once occurred to me that the fumbling...,HPL
2,"In his left hand was a gold snuff box, from wh...",EAP
3,How lovely is spring As we looked from Windsor...,MWS
4,"Finding nothing else, not even gold, the Super...",HPL
5,"A youth passed in solitude, my best years spen...",MWS
6,"The astronomer, perhaps, at this point, took r...",EAP
7,The surcingle hung in ribands from my body.,EAP
8,I knew that you could not say to yourself 'ste...,EAP
9,I confess that neither the structure of langua...,MWS


In [0]:
# exploring.
'''

let us now see some worko f each author and see if we can figure out any patterns in their writing.

'''

eap_df=df.loc[df.author=='EAP','text']

In [15]:
print("Author EAP:","\n")
for i,text in enumerate(eap_df):
  print(text,"\n")
  if(i==4):                 # no conclusions.
    break

Author EAP: 

This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall. 

In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction. 

The astronomer, perhaps, at this point, took refuge in the suggestion of non luminosity; and here analogy was suddenly let fall. 

The surcingle hung in ribands from my body. 

I knew that you could not say to yourself 'stereotomy' without being brought to think of atomies, and thus of the theories of Epicurus; and since, when we discussed this subject not very long ago, I mentioned to you how singularly, yet with how little notice, the vague guesses of that noble Greek had met with confirmation in the late nebular cosmogony, I felt that you c

In [16]:
hpl_df=df.loc[df.author=='HPL','text']
print("Author HPL:","\n")
for i,text in enumerate(hpl_df):
  print(text,"\n")
  if(i==4):                 # no conclusions.
    break

Author HPL: 

It never once occurred to me that the fumbling might be a mere mistake. 

Finding nothing else, not even gold, the Superintendent abandoned his attempts; but a perplexed look occasionally steals over his countenance as he sits thinking at his desk. 

Herbert West needed fresh bodies because his life work was the reanimation of the dead. 

The farm like grounds extended back very deeply up the hill, almost to Wheaton Street. 

His facial aspect, too, was remarkable for its maturity; for though he shared his mother's and grandfather's chinlessness, his firm and precociously shaped nose united with the expression of his large, dark, almost Latin eyes to give him an air of quasi adulthood and well nigh preternatural intelligence. 



In [17]:
mws_df=df.loc[df.author=='MWS','text']
print("Author MWS:","\n")
for i,text in enumerate(mws_df):
  print(text,"\n")
  if(i==4):                 # no conclusions.
    break

Author MWS: 

How lovely is spring As we looked from Windsor Terrace on the sixteen fertile counties spread beneath, speckled by happy cottages and wealthier towns, all looked as in former years, heart cheering and fair. 

A youth passed in solitude, my best years spent under your gentle and feminine fosterage, has so refined the groundwork of my character that I cannot overcome an intense distaste to the usual brutality exercised on board ship: I have never believed it to be necessary, and when I heard of a mariner equally noted for his kindliness of heart and the respect and obedience paid to him by his crew, I felt myself peculiarly fortunate in being able to secure his services. 

I confess that neither the structure of languages, nor the code of governments, nor the politics of various states possessed attractions for me. 

He shall find that I can feel my injuries; he shall learn to dread my revenge" A few days after he arrived. 

He had escaped me, and I must commence a destruct

In [0]:
# loading pre-trained google word2vec embeddings.
from gensim.models import KeyedVectors
w2v_model_pre = KeyedVectors.load_word2vec_format(r'drive/Colab Notebooks/spooky author/GoogleNews-vectors-negative300.bin', binary=True)

In [19]:
# try to seek word embedding of a random word.
w2v_model_pre.wv.get_vector('lovely')

array([ 0.00060272,  0.06176758, -0.125     ,  0.171875  ,  0.171875  ,
       -0.078125  ,  0.00787354, -0.02832031, -0.15039062,  0.26171875,
       -0.17089844, -0.22167969, -0.09326172, -0.10107422,  0.17382812,
        0.16308594,  0.08154297,  0.12695312, -0.11279297, -0.02868652,
       -0.00421143,  0.16894531,  0.16308594, -0.08251953,  0.28515625,
        0.17675781, -0.03955078,  0.11328125,  0.01409912, -0.18359375,
       -0.17480469,  0.05981445,  0.24804688,  0.1875    , -0.03588867,
       -0.29882812,  0.14257812, -0.12988281, -0.17871094,  0.15625   ,
        0.04785156, -0.20605469,  0.34765625,  0.02587891,  0.21582031,
       -0.07421875,  0.22363281,  0.02111816, -0.01519775, -0.02319336,
        0.05029297,  0.07568359,  0.26367188,  0.15039062, -0.12060547,
        0.00598145, -0.07666016, -0.08496094, -0.02001953, -0.01275635,
       -0.18164062,  0.10644531, -0.20898438, -0.265625  ,  0.03759766,
        0.08496094, -0.18457031, -0.08789062, -0.11083984,  0.32

In [20]:
'''

note that i am not using this pretarained model anymore beacuse there are many words 
herer that are not in the pretrained model. may be this is because those words didnt occur in the training corpus ie google news corpus.

one such word is 'foster' some thing and this showed error for this.


now after reading on net there are a couple of things which one can do but none of them makes sense to me.

the first one is to leave thiose words which are not in the pre-trained vocab but actually those words are mlore importanrt as those words are specific to our task.

the second option given on net is to use a fixed random vector to evry unknown word.
now that in MY ppoint of view will lead to overfitting as we are making some useless pattern (fixed vector for every word).

Anyways each of this approach can be easily tried and implemented if u want.

'''

"\n\nnote that i am not using this pretarained model anymore beacuse there are many words \nherer that are not in the pretrained model. may be this is because those words didnt occur in the training corpus ie google news corpus.\n\none such word is 'foster' some thing and this showed error for this.\n\n\nnow after reading on net there are a couple of things which one can do but none of them makes sense to me.\n\nthe first one is to leave thiose words which are not in the pre-trained vocab but actually those words are mlore importanrt as those words are specific to our task.\n\nthe second option given on net is to use a fixed random vector to evry unknown word.\nnow that in MY ppoint of view will lead to overfitting as we are making some useless pattern (fixed vector for every word).\n\nAnyways each of this approach can be easily tried and implemented if u want.\n\n"

In [21]:
'''

now let us try to train the embeddings on ourc orpus like we did in imdb dataset.

'''

'\n\nnow let us try to train the embeddings on ourc orpus like we did in imdb dataset.\n\n'

In [0]:
# pre-processing the data as needed by Gensim. (that 'sentences' arguement)

In [0]:
import re

def clean_text(text):
  text = re.sub("[^a-zA-Z]"," ",text)
  # 3. Converting to lower case and splitting
  word_tokens=text.lower().split()
    
    # 4. Remove stopwords
  stop_words= set(stopwords.words("english"))     
  word_tokens= [w for w in word_tokens if not w in stop_words]
    
  cleaned_review=" ".join(word_tokens)
  return cleaned_review
   

In [0]:
# time taking cell.
 
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences=[]
for text in train['text']:
  # print(type(review))
  sents=tokenizer.tokenize(str(text).strip())
  for sent in sents:
    cleaned_sent=clean_text(sent)
    sentences.append(cleaned_sent.split())
 
 

In [0]:
 # print(len(sentences))

In [0]:
# adding sentences from test set so that also see the vocabulary in test set.
for text in test['text']:
  # print(type(review))
  sents=tokenizer.tokenize(str(text).strip())
  for sent in sents:
    cleaned_sent=clean_text(sent)
    sentences.append(cleaned_sent.split())

In [27]:
print(len(sentences))

28834


In [28]:
test['text']

0       Still, as I urged our leaving Ireland with suc...
1       If a fire wanted fanning, it could readily be ...
2       And when they had broken down the frail door t...
3       While I was thinking how I should possibly man...
4       I am not sure to what limit his knowledge may ...
5       "The thick and peculiar mist, or smoke, which ...
6       That which is not matter, is not at all unless...
7       I sought for repose although I did not hope fo...
8       Upon the fourth day of the assassination, a pa...
9              "The tone metaphysical is also a good one.
10      These, the offspring of a later period, stood ...
11      What kept him from going with her and Brown Je...
12      Persuading the widow that my connexion with he...
13      When I arose trembling, I know not how much la...
14      And by the shores of the river Zaire there is ...
15      Idris heard of her mother's return with pleasure.
16      I say this proudly, but with tears in my eyes ...
17      But le

In [0]:
w2v_model=Word2Vec(sentences,size=300,window=10) # try to tune parameters.

In [30]:
w2v_model.train(sentences,total_examples=len(sentences),epochs=10)

(3292547, 3643640)

In [31]:
w2v_model

In [0]:
df['text']=df['text'].apply(clean_text)
test['text']=test['text'].apply(clean_text)

In [0]:
def make_doc_vectors(review):
  doc_vec=np.zeros(300,dtype='float64') # initialize with zeros.
  words=review.split()
  for word in words: # iterate over all words.
    if word in w2v_model.wv.vocab:
      word_vec=w2v_model.wv.get_vector(word) # if word in vocab add its word vector to our doc vector
      doc_vec=doc_vec+word_vec
  return doc_vec/len(words)  

In [0]:
doc_vectors_train=[]
for review in df['text']:
  val=make_doc_vectors(review)
  doc_vectors_train.append(val)
doc_vectors_train=np.array(doc_vectors_train)

In [59]:
doc_vectors_train

array([[-0.06558332, -0.16856991,  0.2569005 , ...,  0.19822277,
         0.4968006 , -0.21380241],
       [-0.15052178,  0.03866731,  0.38813798, ...,  0.48441719,
         0.54545498, -0.03405831],
       [ 0.08852132, -0.14487087,  0.05528754, ..., -0.054616  ,
         0.22446379, -0.11889364],
       ...,
       [ 0.02334363,  0.13543085,  0.11341397, ...,  0.23190454,
         0.24117448,  0.0506095 ],
       [-0.05008939, -0.26478426, -0.05355897, ...,  0.00123336,
         0.2369076 , -0.03962618],
       [ 0.02919731,  0.02457661, -0.01034997, ...,  0.03794237,
         0.17442343,  0.04021765]])

In [0]:
le=LabelEncoder()
Y=le.fit_transform(df['author'])

In [61]:
x_train,x_test,y_train,y_test=train_test_split(doc_vectors_train,Y,test_size=0.2,random_state=42)
np.isnan(x_train)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [62]:
doc_vectors_train

array([[-0.06558332, -0.16856991,  0.2569005 , ...,  0.19822277,
         0.4968006 , -0.21380241],
       [-0.15052178,  0.03866731,  0.38813798, ...,  0.48441719,
         0.54545498, -0.03405831],
       [ 0.08852132, -0.14487087,  0.05528754, ..., -0.054616  ,
         0.22446379, -0.11889364],
       ...,
       [ 0.02334363,  0.13543085,  0.11341397, ...,  0.23190454,
         0.24117448,  0.0506095 ],
       [-0.05008939, -0.26478426, -0.05355897, ...,  0.00123336,
         0.2369076 , -0.03962618],
       [ 0.02919731,  0.02457661, -0.01034997, ...,  0.03794237,
         0.17442343,  0.04021765]])

In [64]:
clf=LogisticRegression()
clf.fit(x_train,y_train)
pred=clf.predict_proba(x_test)
print(log_loss(y_test,pred))

ValueError: ignored

In [0]:
nan_x=np.isnan(x_train)

In [56]:
'''

on running LR it is showing that input has nan values. i have checked everything but i am not able to figure out whats the cause.

hence now i am leaving this as I know how to cretae the embeddings and how to represent document vectors using this method of averaged w2v vectors and also seen how to load and create pre-trained embeddings. this is takeway from this notebook.

'''

'\n\non running LR it is showing that input has nan values. i have checked everything but i am not able to figure out whats the cause.\n\nhence now i am leaving this as I know how to cretae the embeddings and how to represent document vectors using this method of averaged w2v vectors and also seen how to load and create pre-trained embeddings. this is takeway from this notebook.\n\n'

In [67]:
x_train.shape

(15663, 300)